In [1]:
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.layers import Normalization
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Flatten
from keras import layers
pd.set_option('display.max_rows', 50)
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping 

import random


2021-09-02 09:44:35.719280: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-02 09:44:35.719330: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
df_merged = pd.read_csv("rnn_model_df_31.csv")

In [3]:
df_merged = df_merged[df_merged["reply_count"].notna()].drop(columns = ["reply_count", "avg_followers_count", "avg_following_count" ] )

In [4]:
df_merged

,tweet_date,party,poll,retweet_count,like_count,avg_len_of_tweet,avg_user_tweet_count,avg_ff_ratio,share_of_tweets,share_unique_users,weighted_sentiment,share_of_positive_tweets,share_of_negative_tweets
420,2021-07-23,AFD,11.00,7430.0,50589.0,183.483988,30258.523792,11.717405,0.241723,0.599421,-0.822698,0.028768,0.587842
421,2021-07-23,CDU,28.00,8987.0,53965.0,191.227455,26716.583105,5.530039,0.207085,0.673073,-0.466696,0.035058,0.480465
422,2021-07-23,FDP,12.00,1854.0,15224.0,171.541570,26521.715935,5.561241,0.113623,0.675905,-0.385271,0.061971,0.476905
423,2021-07-23,GRUENE,19.00,7621.0,60041.0,182.306444,24578.680470,4.759415,0.264028,0.619016,-0.267626,0.060460,0.439125
424,2021-07-23,LINKE,7.00,268.0,1625.0,181.312292,36722.935216,2.647902,0.026328,0.616279,-0.050804,0.061462,0.373754
...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,2021-08-31,FDP,12.00,2626.0,30829.0,148.846072,20007.450027,3.306954,0.112983,0.721005,-0.545316,0.051131,0.523784
696,2021-08-31,GRUENE,17.50,4733.0,38891.0,180.245516,23105.896824,6.798644,0.122323,0.688827,-0.115527,0.063847,0.393780
697,2021-08-31,LINKE,6.50,1411.0,10948.0,176.158121,27386.891000,2.613547,0.056139,0.547508,-0.117520,0.057727,0.389745
698,2021-08-31,OTHER,7.67,1580.0,9130.0,164.704657,16861.637255,2.676769,0.032850,0.670343,0.000828,0.088848,0.371936


In [5]:
df_merged = df_merged.replace(np.nan, 0)

In [7]:
df_merged["tweet_date"] = pd.to_datetime(df_merged["tweet_date"])

df_merged = df_merged.groupby([pd.Grouper(key='tweet_date',freq='D'), "party"]).agg({
#     "reply_count": "sum", 
    "retweet_count": "sum",
    "like_count": "sum",
#     "avg_len_of_tweet": "mean" ,
#     "avg_followers_count": "mean" ,
#     "avg_following_count": "mean" ,
    "avg_user_tweet_count" : "mean",
    "avg_ff_ratio" :"mean",
    "share_of_tweets" :"mean",
    "share_unique_users": "mean",
    "weighted_sentiment" : "mean",
    "share_of_positive_tweets": "mean",
    "share_of_negative_tweets" : "mean",
    "poll": "mean"}) 

In [8]:
df_real_true = df_merged.iloc[-7:]

In [9]:
df_merged = df_merged.iloc[:-7]

In [10]:
df_merged_final_test = df_merged.iloc[-7:]

In [11]:
df_merged_final = df_merged.iloc[:-7]

In [12]:
num_features = [ "retweet_count", "like_count", "avg_user_tweet_count","avg_ff_ratio"]
remaing_columns = ["share_of_tweets","share_unique_users","weighted_sentiment","share_of_positive_tweets","share_of_negative_tweets"]

In [13]:
from sklearn.preprocessing import MinMaxScaler

mm_scaler = MinMaxScaler()

mm_scaler.fit(df_merged_final[num_features])

numerical_train_scaled = mm_scaler.transform(df_merged_final[num_features])
numerical_test_scaled = mm_scaler.transform(df_merged_final_test[num_features])

In [14]:
df_merged_poll = df_merged_final["poll"] / 100
df_merged_poll_test = df_merged_final_test["poll"] / 100

In [15]:
df_numerical_train_scaled = pd.DataFrame(numerical_train_scaled, columns = num_features, index = df_merged_final.index)


In [16]:
df_train_scaled = pd.concat([df_numerical_train_scaled,df_merged_final[remaing_columns], pd.DataFrame(df_merged_poll, columns = ["poll"], index = df_merged_final.index)] , axis = 1)

In [17]:
df_test_scaled = pd.concat([pd.DataFrame(numerical_test_scaled, columns = num_features, index = df_merged_final_test.index),df_merged_final_test[remaing_columns], pd.DataFrame(df_merged_poll_test, columns = ["poll"], index = df_merged_final_test.index)] , axis = 1)

In [18]:
def subsequence(df, length, start = 0):
    last_possible_start = len(df) - length 
    X = df[start:start + length]
    y = df["poll"][(start + length ) : (start + length + 7)]
    
    return X, y

In [19]:
def multiple_subsequences(df, length):
    
    list_of_X = []
    list_of_y = []
    
    number_of_subsequences = round(len(df) /  7 - (length/7))
    print(number_of_subsequences)
    start = 0
    for num in range(number_of_subsequences):
        temporary_X, temporary_y = subsequence(df, length, start)
        list_of_X.append(temporary_X)
        list_of_y.append(temporary_y)
        start = start + 7
    
    return list_of_X, list_of_y

    
    
    

In [20]:
X_train , y_train = multiple_subsequences(pd.DataFrame(df_train_scaled), 28 )
# X_test , y_test = multiple_subsequences(df_test_scaled, 35)

34


In [21]:
(df_train_scaled.shape[0]/ 7)

38.0

In [22]:
X_train = np.array(X_train)
# X_test = np.array(X_test)
y_train = np.array(y_train)
# y_test = np.array(y_test)


In [23]:
from keras.layers import BatchNormalization


In [24]:
model = Sequential()
# model.add(layers.Masking(mask_value=-9999, input_shape=X_train[0].shape))
model.add(layers.LSTM(units=128, activation='tanh', return_sequences=True))
model.add(layers.LSTM(units=64, activation='tanh', return_sequences=False))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(7, activation='linear'))


2021-09-02 09:44:57.450069: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-02 09:44:57.450133: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-02 09:44:57.450151: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-69AIGBNI): /proc/driver/nvidia/version does not exist
2021-09-02 09:44:57.450411: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [25]:
%%time

es = EarlyStopping(  monitor='val_mae', patience = 50, restore_best_weights=True)


model.compile(loss='mse', optimizer='adam', metrics=['mae'])

history = model.fit(X_train, y_train ,
          epochs=100, 
          batch_size=1, 
          verbose=1, 
          validation_split=0.2,
          shuffle = False,
          callbacks=[es]
     )



2021-09-02 09:44:57.667755: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
27/27 [==============================] - 3s 36ms/step - loss: 0.0039 - mae: 0.0397 - val_loss: 5.3867e-04 - val_mae: 0.0179
Epoch 2/100
27/27 [==============================] - 0s 17ms/step - loss: 2.8434e-04 - mae: 0.0135 - val_loss: 3.2018e-04 - val_mae: 0.0147
Epoch 3/100
27/27 [==============================] - 0s 16ms/step - loss: 4.3327e-04 - mae: 0.0165 - val_loss: 3.9596e-04 - val_mae: 0.0126
Epoch 4/100
27/27 [==============================] - 0s 16ms/step - loss: 2.3938e-04 - mae: 0.0120 - val_loss: 3.4601e-04 - val_mae: 0.0130
Epoch 5/100
27/27 [==============================] - 0s 16ms/step - loss: 1.9823e-04 - mae: 0.0112 - val_loss: 3.4258e-04 - val_mae: 0.0129
Epoch 6/100
27/27 [==============================] - 0s 15ms/step - loss: 1.9521e-04 - mae: 0.0113 - val_loss: 3.3611e-04 - val_mae: 0.0127
Epoch 7/100
27/27 [==============================] - 0s 16ms/step - loss: 1.9450e-04 - mae: 0.0113 - val_loss: 3.3165e-04 - val_mae: 0.0126
Epoch 8/100
27/27 [=====

27/27 [==============================] - 1s 27ms/step - loss: 9.1393e-05 - mae: 0.0073 - val_loss: 1.7185e-04 - val_mae: 0.0106
Epoch 60/100
27/27 [==============================] - 1s 29ms/step - loss: 1.0627e-04 - mae: 0.0078 - val_loss: 1.7897e-04 - val_mae: 0.0109
Epoch 61/100
27/27 [==============================] - 1s 29ms/step - loss: 1.0982e-04 - mae: 0.0079 - val_loss: 1.8429e-04 - val_mae: 0.0110
Epoch 62/100
27/27 [==============================] - 1s 30ms/step - loss: 1.0753e-04 - mae: 0.0078 - val_loss: 1.6241e-04 - val_mae: 0.0099
Epoch 63/100
27/27 [==============================] - 1s 29ms/step - loss: 1.1230e-04 - mae: 0.0079 - val_loss: 1.9114e-04 - val_mae: 0.0110
Epoch 64/100
27/27 [==============================] - 1s 37ms/step - loss: 1.1588e-04 - mae: 0.0081 - val_loss: 2.8172e-04 - val_mae: 0.0121
Epoch 65/100
27/27 [==============================] - 1s 37ms/step - loss: 8.2112e-05 - mae: 0.0067 - val_loss: 2.6892e-04 - val_mae: 0.0122
Epoch 66/100
27/27 [======

In [26]:
df_merged_final_day = np.array(df_test_scaled)
df_merged_final_day = np.expand_dims(df_merged_final_day, axis = 0)

In [27]:
df_prediction = pd.DataFrame(model.predict(df_merged_final_day)).rename( \
                        columns = {0:'AFD',1:'CDU',2:'FDP',3: 'GRUENE',4: 'LINKE', 5:'OTHER', 6: 'SPD'}).round(3)
df_prediction.index = [df_real_true.index[0][0]]

df_prediction

,AFD,CDU,FDP,GRUENE,LINKE,OTHER,SPD
2021-08-31,0.107,0.237,0.111,0.179,0.065,0.074,0.193


In [ ]:
df_merged_final_test.index

In [ ]:
df_prediction_2 = pd.DataFrame((df_prediction.iloc[0]) / np.sum(df_prediction.iloc[0])).T
df_prediction_2 = df_prediction_2.round(2)

In [ ]:
df_prediction_2

In [ ]:
df_prediction_2.to_csv("final_real_dataframe_for_philip.csv")

In [ ]:
df_true = pd.DataFrame(df_real_true["poll"]).T/100

In [ ]:
df_true = pd.DataFrame(df_true.iloc[0]).reset_index().groupby(by = "party").mean().T

In [ ]:
df_true

In [ ]:
df_plotting_spd = df_train_scaled.reset_index().loc[df_train_scaled.reset_index()["party"] == "SPD"].drop(columns = "party")\
[["tweet_date","share_of_tweets", "weighted_sentiment", "share_of_positive_tweets", "share_of_negative_tweets", "poll"]]
df_plotting_cdu = df_train_scaled.reset_index().loc[df_train_scaled.reset_index()["party"] == "CDU"].drop(columns = "party")\
[["tweet_date","share_of_tweets", "weighted_sentiment", "share_of_positive_tweets", "share_of_negative_tweets", "poll"]]


In [ ]:
sns.set(rc={'figure.figsize':(15,10)})

new_df = df_plotting_spd.melt(id_vars='tweet_date', 
                 value_name='val', 
                 var_name='type')

sns.lineplot(x='tweet_date', y='val', hue='type', data=new_df)

In [ ]:
new_df = df_plotting_cdu.melt(id_vars='tweet_date', 
                 value_name='val', 
                 var_name='type')

sns.lineplot(x='tweet_date', y='val', hue='type', data=new_df)

In [ ]:
df_merged_final_test

In [ ]:
5+5